In [24]:
pip install lightgbm optuna

Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
Note: you may need to restart the kernel to use updated packages.


In [1]:
import boto3
import pandas as pd; pd.set_option('display.max_columns', 100)
import numpy as np

import matplotlib.pyplot as plt; plt.style.use('ggplot')
import seaborn as sns

from scipy.stats import rankdata
from sklearn.cluster import KMeans
from sklearn.multiclass import OneVsRestClassifier
from sklearn.tree import DecisionTreeRegressor, DecisionTreeClassifier, plot_tree
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import KFold, train_test_split, GridSearchCV, StratifiedKFold, TimeSeriesSplit
from sklearn.metrics import mean_squared_error, roc_auc_score, cohen_kappa_score, davies_bouldin_score, calinski_harabasz_score, silhouette_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, HistGradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from lightgbm import LGBMClassifier, LGBMRegressor
# from xgboost import XGBClassifier, XGBRegressor
# from catboost import CatBoostClassifier

import optuna 

s3 = boto3.resource('s3')
bucket_name = 'analytics-data-science-competitions'
bucket = s3.Bucket(bucket_name)

file_key_1 = 'Tabular-Playground-Series/TS-S3-Ep5/train.csv'
file_key_2 = 'Tabular-Playground-Series/TS-S3-Ep5/test.csv'
file_key_3 = 'Tabular-Playground-Series/TS-S3-Ep5/sample_submission.csv'

bucket_object_1 = bucket.Object(file_key_1)
file_object_1 = bucket_object_1.get()
file_content_stream_1 = file_object_1.get('Body')

bucket_object_2 = bucket.Object(file_key_2)
file_object_2 = bucket_object_2.get()
file_content_stream_2 = file_object_2.get('Body')

bucket_object_3 = bucket.Object(file_key_3)
file_object_3 = bucket_object_3.get()
file_content_stream_3 = file_object_3.get('Body')

## Reading data files
train = pd.read_csv(file_content_stream_1)
test = pd.read_csv(file_content_stream_2)
submission = pd.read_csv(file_content_stream_3)

## Enginering features
train['alcohol_density'] = train['alcohol'] * train['density']
train['sulphate/density'] = train['sulphates']  / train['density']
train['alcohol_sulphate'] = train['alcohol'] * train['sulphates']

test['alcohol_density'] = test['alcohol']  * test['density']
test['sulphate/density'] = test['sulphates']  / test['density']
test['alcohol_sulphate'] = test['alcohol'] * test['sulphates']

test_md = test.copy()

X = train[['sulphate/density', 'alcohol_density', 'alcohol', 'sulphates']]
Y = train['quality'] 

test_md = test_md[['sulphate/density', 'alcohol_density', 'alcohol', 'sulphates']]

# Optimal Rounder

In [2]:
from functools import partial
import numpy as np
import scipy as sp

class OptimizedRounder(object):
    def __init__(self):
        self.coef_ = 0

    def _kappa_loss(self, coef, X, y):
        X_p = np.copy(X)
        for i, pred in enumerate(X_p):
            if pred < coef[0]:
                X_p[i] = 3
            elif pred >= coef[0] and pred < coef[1]:
                X_p[i] = 4
            elif pred >= coef[1] and pred < coef[2]:
                X_p[i] = 5
            elif pred >= coef[2] and pred < coef[3]:
                X_p[i] = 6
            elif pred >= coef[3] and pred < coef[4]:
                X_p[i] = 7
            else:
                X_p[i] = 8

        ll = cohen_kappa_score(y, X_p, weights = 'quadratic')
        return -ll

    def fit(self, X, y):
        loss_partial = partial(self._kappa_loss, X = X, y = y)
        initial_coef = [3.5, 4.5, 5.5, 6.5, 7.5]
        self.coef_ = sp.optimize.minimize(loss_partial, initial_coef, method = 'nelder-mead')

    def predict(self, X, coef):
        X_p = np.copy(X)
        for i, pred in enumerate(X_p):
            if pred < coef[0]:
                X_p[i] = 3
            elif pred >= coef[0] and pred < coef[1]:
                X_p[i] = 4
            elif pred >= coef[1] and pred < coef[2]:
                X_p[i] = 5
            elif pred >= coef[2] and pred < coef[3]:
                X_p[i] = 6
            elif pred >= coef[3] and pred < coef[4]:
                X_p[i] = 7
            else:
                X_p[i] = 8
        return X_p

    def coefficients(self):
        return self.coef_['x']

# Optuna Optimzation

In [3]:
def objective(trial):
    
    ## Parameters to be evaluated
    param = dict(objective = 'regression_l1',
                 verbose = -100,
                 boosting_type = 'gbdt', 
                 random_state = 42,
                 n_estimators = trial.suggest_int('n_estimators', 300, 10000),
                 learning_rate = trial.suggest_float('learning_rate', 0.001, 1, log=True),
                 max_depth = trial.suggest_int('max_depth', 3, 12),
                 lambda_l1 = trial.suggest_float('lambda_l1', 0.01, 10.0, log=True),
                 lambda_l2 = trial.suggest_float('lambda_l2', 0.01, 10.0, log=True),
                 num_leaves = trial.suggest_int('num_leaves', 2, 100),
                 bagging_fraction = trial.suggest_float('bagging_fraction', 0.2, 0.9),
                 feature_fraction = trial.suggest_float('feature_fraction', 0.2, 0.9)
                 )

    
    scores = []

#     skf = StratifiedKFold(n_splits = 5, shuffle = True, random_state = 42)
    skf = KFold(n_splits = 5, shuffle = True, random_state = 42)
    
    for fold, (train_idx, valid_idx) in enumerate(skf.split(X, Y)):
        
        print(fold, end = ' ')
        X_train, X_valid = X.iloc[train_idx], X.iloc[valid_idx]
        y_train , y_valid = Y.iloc[train_idx] , Y.iloc[valid_idx]

        model = LGBMRegressor(**param)
        model.fit(X_train, y_train)

        preds_valid = model.predict(X_valid)
        
        optR = OptimizedRounder()
        optR.fit(preds_valid, y_valid)
        coef = optR.coefficients()
        preds_valid = optR.predict(preds_valid, coef).astype(int)
        
        score = cohen_kappa_score(y_valid,  preds_valid, weights = "quadratic")
        scores.append(score)
        
    return np.mean(scores)

In [28]:
study = optuna.create_study(direction = "maximize")
study.optimize(objective, n_trials = 50, timeout = 3600)

[I 2023-02-06 20:53:22,549] A new study created in memory with name: no-name-105458ed-b7b8-4a45-8d60-972b8c18b1b0


0 [LightGBM] [Warning] lambda_l1 is set=0.38218629383871416, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.38218629383871416
[LightGBM] [Warning] bagging_fraction is set=0.3319118604868666, subsample=1.0 will be ignored. Current value: bagging_fraction=0.3319118604868666
[LightGBM] [Warning] feature_fraction is set=0.7672166744399376, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.7672166744399376
[LightGBM] [Warning] lambda_l2 is set=0.027010052172435354, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.027010052172435354
1 [LightGBM] [Warning] lambda_l1 is set=0.38218629383871416, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.38218629383871416
[LightGBM] [Warning] bagging_fraction is set=0.3319118604868666, subsample=1.0 will be ignored. Current value: bagging_fraction=0.3319118604868666
[LightGBM] [Warning] feature_fraction is set=0.7672166744399376, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.767

[I 2023-02-06 20:53:29,090] Trial 0 finished with value: 0.3880404541003152 and parameters: {'n_estimators': 6968, 'learning_rate': 0.0030967402634097716, 'max_depth': 11, 'lambda_l1': 0.38218629383871416, 'lambda_l2': 0.027010052172435354, 'num_leaves': 2, 'bagging_fraction': 0.3319118604868666, 'feature_fraction': 0.7672166744399376}. Best is trial 0 with value: 0.3880404541003152.


0 [LightGBM] [Warning] lambda_l1 is set=0.015170233035481328, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.015170233035481328
[LightGBM] [Warning] bagging_fraction is set=0.5377442807200433, subsample=1.0 will be ignored. Current value: bagging_fraction=0.5377442807200433
[LightGBM] [Warning] feature_fraction is set=0.703251472437088, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.703251472437088
[LightGBM] [Warning] lambda_l2 is set=1.253274261270629, reg_lambda=0.0 will be ignored. Current value: lambda_l2=1.253274261270629
1 [LightGBM] [Warning] lambda_l1 is set=0.015170233035481328, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.015170233035481328
[LightGBM] [Warning] bagging_fraction is set=0.5377442807200433, subsample=1.0 will be ignored. Current value: bagging_fraction=0.5377442807200433
[LightGBM] [Warning] feature_fraction is set=0.703251472437088, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.70325147

[I 2023-02-06 20:53:33,155] Trial 1 finished with value: 0.5254418885456448 and parameters: {'n_estimators': 610, 'learning_rate': 0.32549346930959216, 'max_depth': 6, 'lambda_l1': 0.015170233035481328, 'lambda_l2': 1.253274261270629, 'num_leaves': 94, 'bagging_fraction': 0.5377442807200433, 'feature_fraction': 0.703251472437088}. Best is trial 1 with value: 0.5254418885456448.


...

[I 2023-02-06 21:02:14,818] Trial 47 finished with value: 0.5631774476500565 and parameters: {'n_estimators': 1708, 'learning_rate': 0.03569780471633349, 'max_depth': 8, 'lambda_l1': 0.46320820289673226, 'lambda_l2': 0.19023462533545885, 'num_leaves': 68, 'bagging_fraction': 0.6498010133032265, 'feature_fraction': 0.22029736271660963}. Best is trial 39 with value: 0.5709286837925462.


0 [LightGBM] [Warning] lambda_l1 is set=0.28980304381838196, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.28980304381838196
[LightGBM] [Warning] bagging_fraction is set=0.53149979229554, subsample=1.0 will be ignored. Current value: bagging_fraction=0.53149979229554
[LightGBM] [Warning] feature_fraction is set=0.2703811616120403, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.2703811616120403
[LightGBM] [Warning] lambda_l2 is set=0.2605049133399094, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.2605049133399094
1 [LightGBM] [Warning] lambda_l1 is set=0.28980304381838196, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.28980304381838196
[LightGBM] [Warning] bagging_fraction is set=0.53149979229554, subsample=1.0 will be ignored. Current value: bagging_fraction=0.53149979229554
[LightGBM] [Warning] feature_fraction is set=0.2703811616120403, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.270381161612040

[I 2023-02-06 21:02:28,877] Trial 48 finished with value: 0.5686545172007703 and parameters: {'n_estimators': 3993, 'learning_rate': 0.00828498794020625, 'max_depth': 11, 'lambda_l1': 0.28980304381838196, 'lambda_l2': 0.2605049133399094, 'num_leaves': 48, 'bagging_fraction': 0.53149979229554, 'feature_fraction': 0.2703811616120403}. Best is trial 39 with value: 0.5709286837925462.


0 [LightGBM] [Warning] lambda_l1 is set=0.2853887101315789, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.2853887101315789
[LightGBM] [Warning] bagging_fraction is set=0.5298697965942635, subsample=1.0 will be ignored. Current value: bagging_fraction=0.5298697965942635
[LightGBM] [Warning] feature_fraction is set=0.27392396545478515, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.27392396545478515
[LightGBM] [Warning] lambda_l2 is set=0.3556487592536847, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.3556487592536847
1 [LightGBM] [Warning] lambda_l1 is set=0.2853887101315789, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.2853887101315789
[LightGBM] [Warning] bagging_fraction is set=0.5298697965942635, subsample=1.0 will be ignored. Current value: bagging_fraction=0.5298697965942635
[LightGBM] [Warning] feature_fraction is set=0.27392396545478515, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.27392396

[I 2023-02-06 21:02:41,238] Trial 49 finished with value: 0.5699614373224217 and parameters: {'n_estimators': 3744, 'learning_rate': 0.00874629729447718, 'max_depth': 10, 'lambda_l1': 0.2853887101315789, 'lambda_l2': 0.3556487592536847, 'num_leaves': 48, 'bagging_fraction': 0.5298697965942635, 'feature_fraction': 0.27392396545478515}. Best is trial 39 with value: 0.5709286837925462.


# LightGBM Modeling

In [5]:
lgb_cv_scores, lgb_imp = list(), list()
preds = list()

skf = KFold(n_splits = 5, shuffle = True, random_state = 42)
    
for train_ix, test_ix in skf.split(X, Y):
        
    ## Splitting the data 
    X_train, X_test = X.iloc[train_ix], X.iloc[test_ix]
    Y_train, Y_test = Y.iloc[train_ix], Y.iloc[test_ix]
    
    ## Building the model
    lgb_md = LGBMRegressor(n_estimators = 1147,
                            max_depth = 10,
                            learning_rate = 0.018,
                            num_leaves = 51,
                            lambda_l1 = 0.2547,
                            lambda_l2 = 0.09305,
                            bagging_fraction = 0.6074,
                            feature_fraction = 0.2868,
                            random_state = 42).fit(X_train, Y_train)
    lgb_imp.append(lgb_md.feature_importances_)
    
    ## Predicting on X_test and test
    lgb_pred_1 = lgb_md.predict(X_test)
    lgb_pred_2 = lgb_md.predict(test_md)
        
    ## Applying Optimal Rounder (using abhishek approach)
    optR = OptimizedRounder()
    optR.fit(lgb_pred_1, Y_test)
    coef = optR.coefficients()
    lgb_pred_1 = optR.predict(lgb_pred_1, coef).astype(int)
    lgb_pred_2 = optR.predict(lgb_pred_2, coef).astype(int)
        
    ## Computing roc-auc score
    lgb_cv_scores.append(cohen_kappa_score(Y_test, lgb_pred_1, weights = 'quadratic'))
    preds.append(lgb_pred_2)

lgb_cv_score = np.mean(lgb_cv_scores)    
print('The average roc-auc score over 5-folds (run 5 times) is:', lgb_cv_score)

[LightGBM] [Warning] lambda_l1 is set=0.2547, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.2547
[LightGBM] [Warning] bagging_fraction is set=0.6074, subsample=1.0 will be ignored. Current value: bagging_fraction=0.6074
[LightGBM] [Warning] feature_fraction is set=0.2868, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.2868
[LightGBM] [Warning] lambda_l2 is set=0.09305, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.09305
[LightGBM] [Warning] lambda_l1 is set=0.2547, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.2547
[LightGBM] [Warning] bagging_fraction is set=0.6074, subsample=1.0 will be ignored. Current value: bagging_fraction=0.6074
[LightGBM] [Warning] feature_fraction is set=0.2868, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.2868
[LightGBM] [Warning] lambda_l2 is set=0.09305, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.09305
[LightGBM] [Warning] lambda_l1 is set=0.2547, reg_al

In [6]:
lgb_preds_test = pd.DataFrame(preds).mode(axis = 0).loc[0, ]

submission['quality'] = lgb_preds_test.astype(int)
submission.head()

,Id,quality
0,2056,5
1,2057,6
2,2058,6
3,2059,7
4,2060,6


In [7]:
submission['quality'].value_counts()

5    552
6    484
7    334
4      2
Name: quality, dtype: int64

In [8]:
submission.to_csv('LightGBM_Reg_FE_2.csv', index = False)